Ideally, these files should be easy for your team to solve, but difficult for other teams. Recall that the input
files will be pooled together into the evaluation set for Phase II.
Please submit to the Gradescope assignment as a team of you and your partner. You should drag and drop
to upload three files with the following names:

- problem1.in
- problem2.in
- problem3.in

In [147]:
import matplotlib.pyplot as plt
import numpy as np
import math
from random import uniform as rand
from random import sample 


# *** --------- DATA TABLES -------------- ***
''' ITEMS DATA
   ---------------------------------
   |Class|Weight|Cost|Resale Value |
   ---------------------------------
   .
   .
   .
   .
   .
   N
   
   CONTSTAINTS DATA (INCOMPATIBALITY)
   ---------------------------------
   |ITEM1| ITEM2| ..........|ITEM k1|
   ---------------------------------
   .
   .
   .
   .
   .
   T .............................k1
   
'''


'''
   ---------- QUESTIONS TO CONSIDER ------------
   What affects accuracy of choosing optimal items?
   What affects run time of algorithm with respect to choosing P, M?
   What affects run time of algorithm with respect to choosing Contraints, # of Classes?
   Does class ordering matter? # That is should we randomized our classes when creating the input file
   Hows does the distribution of profit/weight affect the accuracy and performance of algorithm?
'''
########################### INPUT FILES ##############################################################
from random import uniform as rand

def randConstrained(n, M):
    splits = [0] + [rand(0, 1) for _ in range(0,n-1)] + [1]
    splits.sort()
    diffs = [x - splits[i - 1] for i, x in enumerate(splits)][1:]
    result = map(lambda x:x*M, diffs)
    return result

'''Creates data for each item:
    N-- Represents the number of items 
    how-- array of length 2 where each entry indicates the way in which we select class, weight, cost & resale value
    P-- weight limit
    M-- money limit
'''
def make_items_data(P, M, N, how):
    data =np.zeros((N,4))
    
    ## ---------------- select ordering of classes ------------ ##
    Class=1
    counter = 1
    if (how[0] == 0): # all different classes
        for i in range(N):
                
            data[i,0]=float(int(i/100))
        
    else:
        raise ValueError('Does not recognize class ordering: ',str(how[0]))
    np.random.shuffle(data)
            
     # *** -------------- select weights, costs & resale value  --------- ***      
            
    if (how[1]==0): #Strongly correlated spanning set
        # Create weights using a randon uniform distribution from 0 to P 
        m= np.random.uniform(low=1, high=10, size=N)
        weights_v = np.random.uniform(low=1, high=P/2, size=2)
        profits_v = weights_v + float(P)/(2.0*10.0)
        max_profit = min(max(profits_v),M)
        costs= np.random.uniform(low=0, high=max_profit , size=N)
        instances = np.random.randint(2, size=N)
        for i in range(N):
            k=instances[i]
            data[i,1]= round(2*weights_v[k]/m[i], 2)
            data[i,2]= round(costs[i], 2)
            data[i,3]= round(2*profits_v[k]/m[i] + costs[i], 2)  #Resale value
            
        
                             
        
        
    elif (how[1]== 1): # price ceiling
        weights = np.random.uniform(low=1, high=P, size=N)
        costs = randConstrained(N,M)
        d = 3
        for i in range(N):
            data[i,1]= round(weights[i], 2)
            data[i, 2]= round(costs[i], 2)
            profit = d*math.ceil(weights[i]/d)
            data[i, 3] = round(profit + costs[i], 2)
            
            
    elif (how[1]== 2): # circle instances
        weights = np.random.uniform(low=1, high=P, size=N)
        d = 2.0/3.0
#       costs = randConstrained(N,M)
        costs = [np.random.uniform(1, M) for _ in range(N)]
        for i in range(N):
            data[i,1]= round(weights[i], 2)
            data[i,2]= round(costs[i], 2)
            profit = d * math.sqrt(4*P**2 - (weights[i] - 2*P)**2)
            data[i,3]= round(profit + costs[i], 2)
        
    else:
        raise ValueError('Does not recognize class ordering: ',str(how[1]))
        
        
    return data
#------ Create Contraints 
'''
    Creates a string with all constraints
    
'''
def make_constraints(C,N,data):
    classes=sample(range(N/100),N/100)
    max_length = 3
    lenghts = np.random.randint(low=2 ,high=max_length, size=C)
    offset=0
    str_constraints=''
    for i in range(C):
        length=lenghts[i]
        for j in range(length):
            if (j+1 == length):
                str_constraints +=str(classes[offset]) +'\n'
            else:
                str_constraints += str(classes[offset])+', '
            offset += 1
    return str_constraints

        


In [179]:
def make_input_file(input_file_name, Pounds, Money, N_items, num_of_constraints, how):
    if (num_of_constraints > 200000):
        raise ValueError('Error constraints be bigger than 200,000')
    f=open(input_file_name + '.in', 'w')
    P=str(Pounds) +'\n'
    M=str(Money) +'\n'
    N=str(N_items)+'\n'
    C=str(num_of_constraints)+'\n'
    f.write(P)
    f.write(M)
    f.write(N)
    f.write(C)
    #-------------------- Creating items and adding them to a file --------------------------------#
    
    Data = make_items_data(Pounds, Money, N_items, how) #Nx4 matrix 
    for i in range(N_items):
        item= str(hex(i+1))[2:]
        line = item + '; ' + str(int(Data[i,0])) + '; ' + str(Data[i,1]) + '; ' + str(Data[i,2]) + '; ' + str(Data[i,3]) + '\n'
        f.write(line)
        
    #---------- Adding Constraints to file ---------------------------------------------------------#
    
    constraints = make_constraints(num_of_constraints, N_items,Data)
    
    f.write(constraints)
        
    f.close() 

In [184]:
make_input_file('problem1', 100, 100, 131000, 150, [0,0])
make_input_file('problem2', 100, 100, 139000, 150, [0,1])
make_input_file('problem3', 100, 100, 124000, 150, [0,2])